In [56]:
import requests
import os
import langid
import threading 

from random import randint
from validators import url as validate
#from math import floor
from bs4 import BeautifulSoup

In [2]:
from urllib.parse import urlparse
def getRootPathFromURL(url):
    url = urlparse(url)
    url = url.scheme + "://" + url.hostname
    return url


In [ ]:
'''

mutual used_urls and urls
open thread
send request
    - add urls
    - add paragraphs
write to file in batches of 32

'''

In [ ]:
usedUrls = []
urls = ["https://en.wikipedia.org/wiki"]
if os.path.exists("text.txt"):
    try:
        file.close()
    except Exception as e:
        print(e)
    os.remove("text.txt")
textFile = open("text.txt","w")

In [147]:
class Crawler:
    
    
    def getText(self,soup=None):
        MIN_LINE_LENGTH = 10
        uniqueText = []
        for line in soup.find_all("p"):
            line = line.get_text()
            line = " ".join(line.split())
            
            isEnglish = langid.classify(line)[0] == "en"
            if isEnglish and not line.endswith(":") and line not in uniqueText and len(line.split()) >= MIN_LINE_LENGTH:
                uniqueText.append(line)
                print(line)
                textFile.write(line + "\n")
            
    def appendURLs(self,soup=None,url=None):
        tempURLs = []
        for href in soup.find_all('a', href=True): # href=True just gets the hrefs from "<a>"s
            URL = getRootPathFromURL(url) + href["href"]
            canAdd = validate(URL) and URL not in urls and URL not in usedUrls and URL not in tempURLs
            if canAdd:
                tempURLs.append(URL)
                #print(URL)
        return tempURLs
                
                
    def getStuff(self,urls):
        
        if len(urls) == 0:
            url = "https://en.wikipedia.org/wiki/Special:Random"
        else:
            url = urls.pop(0)
        
        if not validate(url):
            return
        
        got = requests.get( url )
        soup = BeautifulSoup(got.content, 'html.parser')
        usedUrls.append(url)
        
        
        try:
            urls.extend(self.appendURLs(soup,url))
            
            self.getText(soup)

        except Exception as e:
            print(e)
            
            
    def __init__(self):
        self.thread = threading.Thread(target=self.getStuff, args=(urls,))
        self.thread.start()
    

In [ ]:
NUM_THREADS = 10000

crawlers = [0] * NUM_THREADS

for index, item in enumerate(crawlers):
    crawlers[index] = Crawler()

James B. Weaver (1833–1912) was a two-time candidate for US president and a congressman from Iowa. After serving in the Union Army in the Civil War, Weaver worked for the election of Republican candidates, but switched to the Greenback Party in 1877, and won election to the House in 1878. The Greenbackers nominated Weaver for president in 1880, but he received only 3.3 percent of the popular vote. He was again elected to the House in 1884 and 1886, where he worked for expansion of the money supply and for the opening of Indian Territory to white settlement. As the Greenback Party fell apart, he helped organize a new left-wing party, the Populists, and was their nominee for president in 1892. This time he gained 8.5 percent of the popular vote and won five states. Several of Weaver's political goals became law after his death, including the direct election of senators and a graduated income tax. (This article is part of a featured topic: 1880 United States presidential election.)
There 

In [136]:
print(len(urls))

2417


In [ ]:
print(open("text.txt","r").read())